In [1]:
from dataset import TextDataset
from torch.utils.data import DataLoader

In [2]:
dataset = TextDataset(path='dataset/train.csv')

========> LOADING DATASET <========


In [3]:
dataset[2]

{'string': 'Title: PRIKNIK Horn Red Electric Air Horn Compressor Interior Dual Tone Trumpet Loud Compatible with SX4, Bullet Points: [Loud Dual Tone Trumpet Horn, Compatible With SX4,Electric Air Horn with Compressor, Compatible With SX4,High Quality, High Intensity Horn Sound, Compatible With SX4,Color: Red, Compatible With SX4,Professional installation required, Compatible With SX4], Description: Specifications: Color: Red, Material: Aluminium, Voltage: 12V, dB: 130 dB (around), Material: Aluminum Pump Head + Steel Pump Body + ABS Shell and Parts DB output: 130db Voltage: 12v Sound Type: Dual Tone Application: 12V Voltage Vehicles With Battery Above 20A Package included: 1 x Dual Tone Air Horn Compatible With SX4',
 'type_id': 7537,
 'length': 748.0314953}

In [4]:
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [5]:
batch = next(iter(loader))
batch

{'string': ['Title: Apocalyptic Commentary, Daniel & Revelation, Bullet Points: nan, Description: nan',
  'Title: Compatible Case for Samsung Galaxy A6 (2018) SM-A600 TRI-Shield Tough Series Rugged Kickstand Cover with Belt Clip Holster & Lanyard [Shock Resistant] for A6 (CAMO Branches), Bullet Points: [BELT CLIP: 360 Degree Rotating Swivel Holster Clip. Clip to your gym bag, backpack, belt or purse.,Rugged Case with inner TPU & Reinforced Corners Offers Protection From Shock/Impact.,Textured Coating and Grooved Edges Provide Enhanced Tactile Grip. Fingerprint Resistant cover with Anti-Slip Design. Reinforced 4 corner protection for resilience.,Includes Free Bonus Lanyard Strap for safe carrying (detachable). Features Built-In Magnetic Kickstand So You Can Prop Phone Horizontal and Watch Videos Hands-Free. Fold kickstand back into the case when not needed.,Precise Cutouts for your headphone jack, charger, speaker, and mic. ONLY FITS GALAXY A6], Description: Elegantly designed for Samsu

In [10]:
batch.keys()

dict_keys(['title', 'id', 'type_id', 'length'])

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('dataset/train.csv')

In [14]:
product_id, title, bullet_points, description, type_id, length = df.iloc[3]

In [15]:
product_id, title, bullet_points, description, type_id, length

(1594019,
 "ALISHAH Women's Cotton Ankle Length Leggings Combo of 2, Plus 12 Colors_L",
 '[Made By 95%cotton and 5% Lycra which gives you 4 ways stretchable comfort, GSM 180 to 190,Come with Elasticized Waist Band, Slim Fit,Bubble free fabric and High quality steaching, Bio Wash,Skin-friendly fabric,ANKLE LENGTH LEGGINGS PACK OF 2]',
 "AISHAH Women's Lycra Cotton Ankel Leggings. Brand: ALISHAH Length: Ankel Length Leggings. Size Chart Medium: 28 - 30 inch Large: 30 - 32 inch X-Large: 32 - 34 inch 2X-Large: 34 - 38 inch The color fade will not be there in washing. This cotton stretch leggings is designed to provide absolute comfort and body fit. Pair it with short or long kurta for modern look. High on style, fit and finish, this leggings is sure to lend you a sophisticated look. Please wash the dark colors separately in order to improve the product life. Disclaimer: Product color may slightly vary due to photographic lighting sources or your monitor settings. Please order according to 

In [10]:
len(df)

2249698

In [11]:
train = df.sample(frac=0.8)
test = df.drop(train.index)
train.to_csv('dataset/split_train.csv', index=False)
test.to_csv('dataset/split_test.csv', index=False)

In [12]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')

c:\Users\Pranav\miniconda3\envs\amazonml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pranav\miniconda3\envs\amazonml\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pranav\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see 

In [15]:
inp = tokenizer(
    "Title: This is a title, Bullet Points: This is a bullet point, Description: This is a description",
    return_tensors="pt",
    padding="max_length",
    truncation=True,
)
out = model(**inp)

In [17]:
out.last_hidden_state.shape

torch.Size([1, 512, 768])

In [18]:
import numpy as np

In [20]:
e = np.load('data/bert_base_uncased_train_embeddings.npy')

array([-5.84851325e-01,  2.69485652e-01, -3.48566562e-01, -2.84460392e-02,
       -2.89732695e-01, -5.74535951e-02,  2.87246287e-01,  2.18627602e-01,
       -5.08042276e-01, -3.07704628e-01, -6.18308149e-02,  9.40529481e-02,
       -1.86465122e-02,  2.61477470e-01,  3.22088122e-01,  2.78569818e-01,
       -2.66845375e-01,  1.51833102e-01,  2.78792918e-01, -2.91695327e-01,
       -6.69493675e-02, -3.38568658e-01, -1.10663310e-01, -3.61440837e-01,
       -1.41224384e-01,  3.23496610e-02, -2.16252953e-01, -1.11684971e-01,
       -3.58390242e-01,  2.25115139e-02, -3.46034467e-01,  2.65614130e-02,
       -2.33613282e-01, -9.20312330e-02,  6.67938590e-01,  2.63723671e-01,
        4.93110530e-02,  1.03723422e-01,  3.89308244e-01,  2.34346434e-01,
        1.86556801e-01,  1.16017357e-01,  1.87476322e-01, -6.79863095e-02,
       -2.35508308e-01, -5.75801060e-02, -3.08353662e+00,  2.81180471e-01,
       -6.39680982e-01, -1.69665083e-01,  3.59200001e-01, -5.08780003e-01,
        4.05128181e-01,  